## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
T_{disk}^4 = \frac{3GM_*\dot{M}}{8\pi\sigma r^3}\left(1-\sqrt{\frac{R_*}{r}}\right)
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


In [98]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [99]:
#Natural Constants

# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [100]:
#Disk Parameters

alpha1 = 1.0 * 10**-2
alpha2 = 1.0 * 10**-3
alpha3 = 1.0 * 10**-4
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2;
distance = rout - rin #distance from inner radius to outer radius
Rstar = 1 * AU
Mdot = 10**-7/yrs2sec#accretion rate in solar mass/sec

In [101]:
#Temporal discretization

dt = 0.5 * yrs2sec #timestep
final_time = 10*yrs2sec #total diffusion time in seconds

In [102]:
#Spacial discretization

n = 100; #number of steps / number of points to plot
dr = distance/n #distance between each spacial step

In [103]:
#Plotting axes
x = np.linspace(rin + dr/2, rout - dr/2, n)
t = np.arange(0, final_time, dt)
points_range = np.arange(0, len(t), n)
time_plot = np.arange(0, final_time, final_time/n)

In [104]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
nu = np.empty(n)
X = np.empty(n)
D = np.empty(n)
sigma = np.empty(n)
df_dt = np.empty(n)
f = np.empty(n)
sigma_evol = np.empty(n)
temp_evol = np.empty(n)
cs2 = np.empty(n)
mass_evol = []

In [105]:
#Boundary Conditions

f_in = 0
f_out = 0

In [106]:
# @numba.njit
def calc_init_params(alpha):
    """
    Calculate the initial parameters values for time t = 0 and changing radius.
    """
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma[i] = sigma_in * (AU) / dist[i]
        if (sigma[i]>sigma_max):
            sigma[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma[i] = sigma_min
        temp_evol[i] = 500
#         temp_evol[i] = (((3*G*Ms*Mdot)/(8*np.pi*sb*dist[i]**3))*(1-np.sqrt(rin/dist[i])))**(1/float(4)) #i got this eq from armitage but im unsure of this is right bc its a steady state soln.
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
        cs2[i] = (kB * temp_evol[i])/(mu*mH)
        nu[i] = alpha*cs2[i]/Omega[i]
        D[i] = 12 * nu[i] / (X[i] ** 2)
        f[i] = (1.5 * X[i] * sigma[i])

In [107]:
# @numba.njit
def calc_sigma_evol(f, alpha):
    total_sigma_evol = [] # surface density for each r at all times
    calc_init_params(alpha)
    for i in range(1, len(t) + 1):
        plt.clf
        for j in range(1, n-1):
            dX1 = X[j] - X[j-1]
            dX2 = X[j+1] - X[j]
            D1 = 0.5 * (D[j] + D[j-1])
            D2 = 0.5 * (D[j+1] + D[j])
            df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
        dX_final = X[-1]-X[-2]
        dX_in = X[1]-X[0]
        df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
        df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
        f = f + df_dt * dt
        sigma_evol = [2*f[k]/(3*X[k]) for k in range(n)] # surface density for each r at time = i
        total_sigma_evol.append(sigma_evol)
    return total_sigma_evol

In [108]:
# @numba.njit
def calc_press_evol(f, alpha):
    """Calculate the pressure evolution at each dt at each  r"""
    sigma_evol = calc_sigma_evol(f, alpha) #surface density at each r for all dt
    press_evol = [[0 for i in range(n)] for j in range(len(sigma_evol))]
    for i in range(len(sigma_evol)):
        for j in range(n):
            press_evol[i][j] = np.sqrt(cs2[j])*sigma[j]*Omega[j]
    return press_evol

In [109]:
def calc_mass_evol(f, alpha):
    total_sigma_evol = calc_sigma_evol(f, alpha)
    mass_evol = np.empty(len(total_sigma_evol))
    disk_mass = 0
    for i in range(len(total_sigma_evol)):
        for j in range(n):
            disk_mass += 2 * np.pi * dist[j] * dr * total_sigma_evol[i][j] #mass of disk at t=i
            mass_evol[i] = disk_mass
            disk_mass = 0
    return mass_evol

In [110]:
def plot_sigma_evol(f, alpha):
    %matplotlib
    total_sigma_evol = calc_sigma_evol(f, alpha)
    for i in range(len(total_sigma_evol)):
        sigma_at_time = total_sigma_evol[i]
        plt.figure(1)
        plt.plot(x/AU, sigma_at_time)
        plt.title('Surface Density Evolution of Protoplanetary Disk')
        plt.xlabel('Radius (AU)')
        plt.ylabel('Surface Density')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()
        plt.pause(0.01)

In [111]:
def plot_mass_evol(f, alpha):
    %matplotlib
    mass_evol = calc_mass_evol(f, alpha)
    plt.figure(2)
    plt.plot(t/yrs2sec, mass_evol)
    plt.title('Conservation of Mass of an Evolving Protoplanetary Disk')
    plt.xlabel('Time (yrs)')
    plt.ylabel('Total Mass (kg)')
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    print(f'The mass of the disk is {mass_evol[0]}\n')

In [112]:
def plot_press_evol(f, alpha):
    %matplotlib
    total_press_evol = calc_press_evol(f, alpha)
    for i in range(len(total_press_evol)):
        press_at_time = total_press_evol[i]
        plt.figure(3)
        plt.plot(x/AU, press_at_time)
        plt.title('Pressure Evolution of Protoplanetary Disk')
        plt.xlabel('Radius (AU)')
        plt.ylabel('Pressure')
        plt.yscale('log')
        plt.show()
        plt.pause(0.01)

In [113]:
def plot_temp_evol():
    %matplotlib
    plt.figure(4)
    plt.plot(x/AU, temp_evol)
    plt.title('Temperature Evolvution in Protoplanetary Disk')
    plt.xlabel('radius (AU)')
    plt.ylabel('Temperature (K)')
    plt.yscale('log')
    plt.show()

In [114]:
plot_sigma_evol(f, alpha2)

Using matplotlib backend: MacOSX


In [115]:
plot_press_evol(f, alpha2)

Using matplotlib backend: MacOSX


In [116]:
plot_temp_evol()

Using matplotlib backend: MacOSX


In [117]:
plot_mass_evol(f, alpha2)

Using matplotlib backend: MacOSX
The mass of the disk is 1.2182717015378666e+26



In [118]:
# def plot_time_evol(f, df_dt, dist, alpha):
#     %matplotlib
#     for i in range(1, len(t) + 1):
#         plt.clf
#         for j in range(1, n-1):
#             dX1 = X[j] - X[j-1]
#             dX2 = X[j+1] - X[j]
#             D1 = 0.5 * (D[j] + D[j-1])
#             D2 = 0.5 * (D[j+1] + D[j])
#             df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
#         dX_final = X[-1]-X[-2]
#         dX_in = X[1]-X[0]
#         df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
#         df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
#         f = f + df_dt * dt
#         sigma_evol = [2*f[k]/(3*X[k]) for k in range(n)] #surface density at each radius at time t=i
#         disk_mass = np.sum([2 * np.pi * dist[k] * dr * sigma_evol[k] for k in range(n)]) #mass of disk at t=i
#         mass_evol.append(disk_mass)
#         plt.figure(1)
#         plt.plot(x/AU, sigma_evol)
#         plt.title('Surface Density Evolution of Protoplanetary Disk')
#         plt.xlabel('Radius (AU)')
#         plt.ylabel('Surface Density')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()
#         plt.pause(0.01)
#     plt.figure(2)
#     plt.plot(t/yrs2sec, mass_evol)
#     plt.title('Conservation of Mass of an Evolving Protoplanetary Disk')
#     plt.xlabel('Time (yrs)')
#     plt.ylabel('Total Mass (kg)')
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()
#     print(f'The mass of the disk is {mass_evol[0]}\n')